In [31]:
import numpy as np
import pandas as pd
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense
import tensorflow.compat.v1 as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import Dense, Conv2D, Flatten, Dropout,BatchNormalization,MaxPooling2D


In [17]:
import csv

data = {}
with open('imgs/driver_imgs_list.csv') as f:
    reader = csv.reader(f)
    
    # first row in csv has column headers, we dont want column headers. 
    next(reader)
    
    # now getting our key-value pair
    for row in reader:
        key = row[1]
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[2]]

In [18]:
data.keys()

dict_keys(['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [19]:
distraction_list = list(data.keys())

In [21]:
# so we need to make master_data directory.
# inside it, we create two subdirectories training and testing.

import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')


In [22]:
# inside the training and testing, we need to make 10 new subdirectories related to these distractions.
# so we iterate over each distractions.
for distraction in distraction_list:
    os.mkdir(os.path.join('master_data/training/', distraction))
    os.mkdir(os.path.join('master_data/testing/', distraction))
# we have created subdirectiories inside training and testing but the directories are empty so we need to fill them
# with source directory images.

In [24]:
import shutil as sh
split_size = 0.8

for dis, images in data.items():
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        source = os.path.join('imgs/train',  dis,image)
        dest = os.path.join('master_data/training',   dis)
        sh.copy(source, dest)
    for image in test_images:
        source = os.path.join('imgs/train', dis, image)
        dest = os.path.join('master_data/testing',  dis)
        sh.copy(source, dest)

In [25]:
train_dir = 'master_data/training'
test_dir = 'master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                train_dir,   # directory from which we want to generate our data.
                                target_size = (100, 100),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 128
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                test_dir,   # directory from which we want to generate our data.
                                target_size = (100, 100),  # telling what is the size of the class
                                class_mode =  'categorical',  # we need to tell what is the mode of classification, eg 'binary' 
                                batch_size = 128
)

Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [27]:
es = EarlyStopping(monitor = 'loss', patience = 2, min_delta=0.01)


In [32]:
model = tf.keras.models.Sequential([
    # layer 1
    Conv2D(16, (3,3), activation = 'relu', input_shape = (100, 100, 3)),
    MaxPooling2D(2,2),
    
    # layer 2
    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    
    #layer 3
    Conv2D(64, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    
    # Dense layer
    Flatten(),
    Dense(1024, activation ='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    # Last Layer is Important coz we have to decide here mow many classes are there in the images
    Dense(10, activation ='softmax')


    
])

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0

In [36]:
model.fit(train_generator, epochs = 1, verbose = 1, validation_data = test_generator, callbacks = [es])

141/141 [==============================] - 884s 6s/step - loss: 0.1952 - accuracy: 0.9435 - val_loss: 2.2563 - val_accuracy: 0.1900
